In [16]:
import numpy as np
import pandas as pd
import re
import string

In [17]:
txt = "Great product. I like it."

In [18]:
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text

In [19]:
with open('../static/model/corpora/stopwords/english', 'r') as file:
    sw = file.read().splitlines()

In [20]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [15]:
def preprocessing(text):
    data = pd.DataFrame([text], columns = ["tweet"])
    data["tweet"] = data["tweet"].apply(lambda x: " ".join(x.lower() for x in x.split()))
    data["tweet"] = data["tweet"].apply(lambda x : " ".join(re.sub(r'^https?:\/\/.*[\r\n]*','',x, flags=re.MULTILINE) for x in x.split()))
    data["tweet"] = data["tweet"].apply(remove_punctuations)
    data["tweet"] = data["tweet"].str.replace('\d+', '', regex=True)
    data["tweet"] = data["tweet"].apply(lambda x: " ".join(x for x in x.split() if x not in sw))
    data["tweet"] = data["tweet"].apply(lambda x : " ".join(ps.stem(x) for x in x.split()))
    return data["tweet"]

In [21]:
preprocessed_txt = preprocessing(txt)

In [22]:
preprocessed_txt

0    great product like
Name: tweet, dtype: object

In [ ]:
vocab = pd.read_csv("../static/model/vocabulary.txt" , header = None)
tokens = vocab[0].tolist()